In [1]:
import pandas as pd
import os
import geopandas as gpd
import plotly.express as px

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_11156\3204584093.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
df_transects = pd.read_csv('00. data/transect_system.csv')
gdf_transects = gpd.GeoDataFrame(df_transects, geometry=gpd.points_from_xy(df_transects.Intersect_lon, df_transects.Intersect_lat)).set_crs('epsg:3587')
gdf_transects = gdf_transects.to_crs('+proj=wintri')
gdf_transects.head(2)

,Unnamed: 0,transect_id,country_id,continent,country_name,changerate,flag_sandy,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,geometry
0,288898,BOX_080_003_0,FRA,Europe,France,-91.82,True,55.308914,-21.229196,55.303230,-21.235681,55.297547,-21.242165,POINT (-15523871.733 2933146.483)
1,288899,BOX_080_003_1,FRA,Europe,France,-0.35,False,55.312392,-21.231845,55.306709,-21.238329,55.301026,-21.244813,POINT (-15523871.728 2933146.484)


In [3]:
gdf_transects.head(2)

,Unnamed: 0,transect_id,country_id,continent,country_name,changerate,flag_sandy,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,geometry
0,288898,BOX_080_003_0,FRA,Europe,France,-91.82,True,55.308914,-21.229196,55.303230,-21.235681,55.297547,-21.242165,POINT (-15523871.733 2933146.483)
1,288899,BOX_080_003_1,FRA,Europe,France,-0.35,False,55.312392,-21.231845,55.306709,-21.238329,55.301026,-21.244813,POINT (-15523871.728 2933146.484)


## Socio-Economic

In [4]:
path_socio = r'P:\11207765-jrc-dems-and-characteriza\Task_2\JRC_deliverable\3_Indicators\GCC_socioeconomic_filtered.csv'

df_socio = pd.read_csv(path_socio)
gdf_worldpop = gpd.GeoDataFrame(df_socio, geometry=gpd.points_from_xy(df_socio.lon, df_socio.lat)).set_crs('epsg:3587')
gdf_socio= gdf_worldpop.to_crs('+proj=wintri')
gdf_socio.head(2)

,id,lon,lat,angles,country,pop_1_m,pop_5_m,pop_10_m,pop_all,roads,railways,ports,airports,gdp_ppp_usd2017_2015,gdp_ppp_grid_2015,geometry
0,BOX_S12E050_0001,51.167463,-10.163451,298.667671,Seychelles,0.0,0.0,0.0,0.0,0,0,0,0,26198.721304,NaN,POINT (-15523879.695 2933152.839)
1,BOX_S12E050_0002,51.164636,-10.172016,293.360252,Seychelles,0.0,0.0,0.0,0.0,0,0,0,0,26198.721304,NaN,POINT (-15523879.695 2933152.830)


In [5]:
gdf_socio = gdf_transects.sjoin_nearest(gdf_socio, how = 'left', max_distance = 1000)

gdf_socio.head(2)

,Unnamed: 0,transect_id,country_id,continent,country_name,changerate,flag_sandy,Start_lon,Start_lat,Intersect_lon,...,pop_1_m,pop_5_m,pop_10_m,pop_all,roads,railways,ports,airports,gdp_ppp_usd2017_2015,gdp_ppp_grid_2015
0,288898,BOX_080_003_0,FRA,Europe,France,-91.82,True,55.308914,-21.229196,55.303230,...,0.0,0.0,0.0,2842.8580,1.0,1.0,0.0,0.0,NaN,27267.73
1,288899,BOX_080_003_1,FRA,Europe,France,-0.35,False,55.312392,-21.231845,55.306709,...,0.0,0.0,0.0,1701.0264,1.0,1.0,0.0,0.0,NaN,27267.73


#### World Population

In [6]:
gdf_worldpop = gdf_socio[['transect_id', 'country_id','continent','country_name','changerate', 'flag_sandy', 'Intersect_lon', 'Intersect_lat', 'pop_1_m', 'pop_5_m', 'pop_10_m', 'pop_all']]

In [7]:

fig = px.scatter_mapbox(
                        gdf_worldpop, lat="Intersect_lat", 
                        lon="Intersect_lon", 
                        hover_name="pop_all", 
                        color= 'pop_all',
                        color_continuous_scale= 'viridis_r',
                        labels = {"pop_all" : "population"},
                        zoom= 4, center = {'lat': 48, 'lon': 5})
fig.update_layout(mapbox_style="open-street-map", margin=dict(l=0,r=0,b=0,t=0), paper_bgcolor="white", showlegend=False,
legend_title_text='Population')
fig.update_traces(marker={'size': 5})

fig.show()

## Gross Domestic Product per capita

In [8]:
gdf_gdp = gdf_socio[['transect_id', 'country_id','continent','country_name','changerate', 'flag_sandy', 'Intersect_lon', 'Intersect_lat', 'gdp_ppp_usd2017_2015', 'gdp_ppp_grid_2015']]

In [9]:
fig = px.scatter_mapbox(
                        gdf_gdp, lat="Intersect_lat", 
                        lon="Intersect_lon", 
                        hover_name="gdp_ppp_grid_2015", 
                        color= 'gdp_ppp_grid_2015',
                        labels = {'gdp_ppp_grid_2015' : 'GDP'},
                        color_continuous_scale= 'viridis_r',
                        zoom= 4, center = {'lat': 48, 'lon': 5})
fig.update_layout(mapbox_style="open-street-map", margin=dict(l=0,r=0,b=0,t=0), paper_bgcolor="white", showlegend=False)
fig.update_traces(marker={'size': 5})

fig.show()

In [10]:
gdf_socio.to_csv('00. data/westeurope_socio.csv')